In [1]:
#@title The Complete "Fresh Start" Training & Testing Notebook (Final Version)

# STEP 1: INSTALL LIBRARIES & IMPORT EVERYTHING
# ===============================================
!pip install -q tensorflow sounddevice librosa PyYAML scikit-learn
import os, glob, yaml, time, random
import numpy as np
import librosa
import sounddevice as sd
from scipy.io.wavfile import write
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, MaxPool2D, LSTM, Dense, Dropout, Bidirectional, Reshape
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
print("✅ Step 1: Libraries are ready.")

# STEP 2: SETUP FOLDERS & CONFIG
# ===============================================
# We will create a new, clean set of folders for this final model
for folder in ["fresh_data/distress", "fresh_data/noise", "public_noise"]:
    os.makedirs(os.path.join("data", folder), exist_ok=True)
os.makedirs("models", exist_ok=True)

config_content = """
audio:
  sample_rate: 16000
  chunk_duration: 2.0
inference:
  distress_threshold: 0.65
"""
with open("config.yaml", "w") as f: f.write(config_content)
with open("config.yaml", 'r') as f: config = yaml.safe_load(f)
os.environ["PATH"] += os.pathsep + r"C:\ffmpeg\bin"
print("✅ Step 2: Fresh folders, config, and FFmpeg path are set.")

# STEP 3: COMPLETE HIGH-QUALITY RECORDING SESSION
# ===============================================
SAMPLE_RATE = config['audio']['sample_rate']
DURATION = config['audio']['chunk_duration']

def record_and_save(label, count, folder_path):
    print(f"\n🎙️ Get ready to record '{label}' {count} times...")
    print("   INSTRUCTIONS: Vary your tone! Say it loudly, softly, quickly, and slowly.")
    time.sleep(2)
    for i in range(count):
        input(f"   Press Enter to start recording sample #{i+1}...")
        print("   Recording...")
        recording = sd.rec(int(DURATION * SAMPLE_RATE), samplerate=SAMPLE_RATE, channels=1)
        sd.wait()
        filename = os.path.join(folder_path, f"rec_{i+1}.wav")
        write(filename, SAMPLE_RATE, recording)
        print(f"   ✅ Saved to {filename}")

# NEW: Record ALL distress sounds into one folder
record_and_save("Scream 'HELP' (with variation)", 15, "data/fresh_data/distress")
record_and_save("Scream 'BACHAO' (with variation)", 15, "data/fresh_data/distress")
record_and_save("A generic loud scream/yell", 10, "data/fresh_data/distress")
record_and_save("Quiet background noise/silence", 30, "data/fresh_data/noise")
print("\n🎉 Step 3: Complete high-quality dataset recorded!")

# STEP 4: DATA PROCESSING & TRAINING
# ===============================================
def load_audio(file_path):
    try: return librosa.load(file_path, sr=SAMPLE_RATE, mono=True)[0]
    except Exception: return None
def to_mel_spectrogram(audio):
    spec = librosa.feature.melspectrogram(y=audio, sr=SAMPLE_RATE, n_mels=128, n_fft=2048, hop_length=512)
    return librosa.power_to_db(spec, ref=np.max)

def process_files(file_paths, label):
    features, labels = [], []
    for f in file_paths:
        audio = load_audio(f)
        if audio is None: continue
        target_len = int(DURATION * SAMPLE_RATE)
        if len(audio) > target_len: audio = audio[:target_len]
        else: audio = np.pad(audio, (0, target_len - len(audio)), 'constant')
        features.append(to_mel_spectrogram(audio)); labels.append(label)
    return features, labels

# --- Creating the Final, Balanced Dataset ---
# POSITIVE DATA: All your new, high-quality distress calls
distress_files = glob.glob("data/fresh_data/distress/*.wav")

# NEGATIVE DATA: Your new quiet room noise + a curated subset of public sounds
public_noise_files = glob.glob("data/public_noise/*.wav")
random.shuffle(public_noise_files)
noise_files = glob.glob("data/fresh_data/noise/*.wav") + public_noise_files[:500]

print(f"--- Creating dataset with {len(distress_files)} distress sounds and {len(noise_files)} noise sounds. ---")

distress_feat, distress_lab = process_files(distress_files, 1)
noise_feat, noise_lab = process_files(noise_files, 0)
X = np.array(distress_feat + noise_feat)[..., np.newaxis]
y = np.array(distress_lab + noise_lab)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

def build_distress_model(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Conv2D(16, (3, 3), activation='relu'), BatchNormalization(), MaxPool2D((2, 2)),
        Conv2D(32, (3, 3), activation='relu'), BatchNormalization(), MaxPool2D((2, 2)),
        Reshape((-1, 448)),
        Bidirectional(LSTM(32)),
        Dense(32, activation='relu'), Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

print("\n--- 🧠 Training the final, definitive model ---")
model = build_distress_model(X_train.shape[1:])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=40, batch_size=16,
          callbacks=[EarlyStopping(monitor='val_accuracy', patience=8, restore_best_weights=True)])

model.save("models/distress_model_1.h5")
print("\n✅ Step 4: Model trained and saved!")

# STEP 5: IMMEDIATE LIVE TEST
# ===============================================
print("\n--- 🔬 Starting IMMEDIATE Live Model Test ---")
THRESHOLD = config['inference']['distress_threshold']
COOLDOWN_SECONDS = 3.0
last_alert_time = 0

def audio_callback(indata, frames, time_info, status):
    global last_alert_time
    if time.time() - last_alert_time < COOLDOWN_SECONDS: return

    mel_spec = to_mel_spectrogram(indata[:, 0].astype('float32'))
    processed_chunk = np.expand_dims(np.expand_dims(mel_spec, axis=0), axis=-1)
    prediction = model.predict(processed_chunk, verbose=0)[0][0]
    
    print(f"Distress Confidence: {prediction:.2f}", end='\r')
    
    if prediction > THRESHOLD:
        print(f"\n\n🚨 SUCCESS! Distress Detected! Confidence: {prediction:.2f} 🚨\n")
        last_alert_time = time.time()

try:
    print(f"\n🎙️  Listening... Scream 'help' or 'bachao'. Threshold is {THRESHOLD}.")
    print("   To stop, click the 'Interrupt the kernel' button (■) in the toolbar above.")
    with sd.InputStream(callback=audio_callback, channels=1, samplerate=SAMPLE_RATE, blocksize=int(DURATION * SAMPLE_RATE)):
        while True: time.sleep(0.1)
except KeyboardInterrupt:
    print("\n⏹️  Stopped listening.")

✅ Step 1: Libraries are ready.
✅ Step 2: Fresh folders, config, and FFmpeg path are set.

🎙️ Get ready to record 'Scream 'HELP' (with variation)' 15 times...
   INSTRUCTIONS: Vary your tone! Say it loudly, softly, quickly, and slowly.


   Press Enter to start recording sample #1... 


   Recording...
   ✅ Saved to data/fresh_data/distress\rec_1.wav


   Press Enter to start recording sample #2... 


   Recording...
   ✅ Saved to data/fresh_data/distress\rec_2.wav


...

   Press Enter to start recording sample #29... 


   Recording...
   ✅ Saved to data/fresh_data/noise\rec_29.wav


   Press Enter to start recording sample #30... 


   Recording...
   ✅ Saved to data/fresh_data/noise\rec_30.wav

🎉 Step 3: Complete high-quality dataset recorded!
--- Creating dataset with 15 distress sounds and 530 noise sounds. ---

--- 🧠 Training the final, definitive model ---
Epoch 1/40
26/26 ━━━━━━━━━━━━━━━━━━━━ 7s 69ms/step - accuracy: 0.9485 - loss: 0.1661 - val_accuracy: 0.9708 - val_loss: 0.1035
Epoch 2/40
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - accuracy: 0.9853 - loss: 0.0396 - val_accuracy: 0.9708 - val_loss: 0.0883
Epoch 3/40
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.9902 - loss: 0.0251 - val_accuracy: 0.9708 - val_loss: 0.1053
Epoch 4/40
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.9926 - loss: 0.0192 - val_accuracy: 0.9708 - val_loss: 0.0718
Epoch 5/40
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.9975 - loss: 0.0155 - val_accuracy: 0.9708 - val_loss: 0.0622
Epoch 6/40
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - accuracy: 1.0000 - loss: 0.0078 - val_accuracy: 0.9781 - val_loss: 0.0560
Epoch


✅ Step 4: Model trained and saved!

--- 🔬 Starting IMMEDIATE Live Model Test ---

🎙️  Listening... Scream 'help' or 'bachao'. Threshold is 0.65.
   To stop, click the 'Interrupt the kernel' button (■) in the toolbar above.
Distress Confidence: 0.86

🚨 SUCCESS! Distress Detected! Confidence: 0.86 🚨

Distress Confidence: 0.98

🚨 SUCCESS! Distress Detected! Confidence: 0.98 🚨

Distress Confidence: 0.79

🚨 SUCCESS! Distress Detected! Confidence: 0.79 🚨

Distress Confidence: 0.93

🚨 SUCCESS! Distress Detected! Confidence: 0.93 🚨

Distress Confidence: 0.93

🚨 SUCCESS! Distress Detected! Confidence: 0.93 🚨

Distress Confidence: 0.98

🚨 SUCCESS! Distress Detected! Confidence: 0.98 🚨

Distress Confidence: 0.94

🚨 SUCCESS! Distress Detected! Confidence: 0.94 🚨

Distress Confidence: 0.95

🚨 SUCCESS! Distress Detected! Confidence: 0.95 🚨

Distress Confidence: 0.67

🚨 SUCCESS! Distress Detected! Confidence: 0.67 🚨

Distress Confidence: 0.00
⏹️  Stopped listening.
